In [ ]:
!pip install youtube-transcript-api # YouTube lib to get transcript
!pip install pytube                 # YouTube lib
!pip install openai                 # OpenAI GPT
!pip install transformers           # Used to know the token size
!pip install chromadb==0.3.2
!pip install python-dotenv
!pip install langchain==0.0.103

In [ ]:
from dotenv import dotenv_values
api_keys = dotenv_values('keys.txt')

import os
os.environ['OPENAI_API_KEY']=api_keys['OPENAI_API_KEY']

In [ ]:
from langchain.llms import OpenAI

llm = OpenAI(openai_api_key=os.environ['OPENAI_API_KEY'])

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=0)

In [ ]:
from langchain.document_loaders.unstructured import UnstructuredFileLoader
from langchain.document_loaders import YoutubeLoader

In [ ]:
loader = YoutubeLoader.from_youtube_url("https://www.youtube.com/watch?v=FtyKgybYkaE", add_video_info=False)
result = loader.load()

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=0)

docs = text_splitter.split_documents(result)
len(docs)

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders import TextLoader

In [ ]:
embeddings = OpenAIEmbeddings()

In [ ]:
db = Chroma.from_documents(docs, embeddings, persist_directory='db')

In [ ]:
query = "How many tickets were sold?"

In [ ]:
search_results = db.similarity_search(query)
len(search_results)

In [ ]:
from langchain.chains.question_answering import load_qa_chain

chain = load_qa_chain(llm, chain_type="stuff")
chain.run(input_documents=search_results, question=query)

In [ ]:
#
# Load QnA 
#
from langchain.chains.qa_with_sources import load_qa_with_sources_chain

chain = load_qa_with_sources_chain(llm, chain_type="stuff")

chain({"input_documents": docs, "question": query}, return_only_outputs=True)

In [ ]:
#
# Load QnA with intermediate steps & map rerank
#
# Map-Rerank:
# This method involves running an initial prompt on each chunk of data, 
# that not only tries to complete a task but also gives a score for how certain it is in its answer. 
# The responses are then ranked according to this score, and the HIGHEST SCORE IS RETURNED.
#
chain = load_qa_with_sources_chain(
    OpenAI(temperature=0), 
    chain_type="map_rerank", 
    metadata_keys=['source'], 
    return_intermediate_steps=True
)

result = chain({"input_documents": docs, "question": query}, return_only_outputs=True)

result["intermediate_steps"]

In [ ]:
len(docs)